![Logo 1](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech1.jpg)
<div class="alert alert-block alert-info">
<h1> Komputerowe wspomaganie tłumaczenia </h1>
<h2> 3. <i>Terminologia</i> [laboratoria]</h2> 
<h3>Rafał Jaworski (2021)</h3>
</div>

![Logo 2](https://git.wmi.amu.edu.pl/AITech/Szablon/raw/branch/master/Logotyp_AITech2.jpg)

Na dzisiejszych zajęciach zajmiemy się bliżej słownikami używanymi do wspomagania tłumaczenia. Oczywiście na rynku dostępnych jest bardzo wiele słowników w formacie elektronicznym. Wiele z nich jest gotowych do użycia w SDL Trados, memoQ i innych narzędziach CAT. Zawierają one setki tysięcy lub miliony haseł i oferują natychmiastową pomoc tłumaczowi.

Problem jednak w tym, iż często nie zawierają odpowiedniej terminologii specjalistycznej - używanej przez klienta zamawiającego tłumaczenie. Terminy specjalistyczne są bardzo częste w tekstach tłumaczonych ze względu na następujące zjawiska:
- Teksty o tematyce ogólnej są tłumaczone dość rzadko (nikt nie tłumaczy pocztówek z pozdrowieniami z wakacji...)
- Te same słowa mogą mieć zarówno znaczenie ogólne, jak i bardzo specjalistyczne (np. "dziedziczenie" w kontekście prawnym lub informatycznym)
- Klient używa nazw lub słów wymyślonych przez siebie, np. na potrzeby marketingowe.

Nietrywialnymi zadaniami stają się: odnalezienie terminu specjalistycznego w tekście źródłowym oraz podanie prawidłowego tłumaczenia tego terminu na język docelowy

Brzmi prosto? Spróbujmy wykonać ręcznie tę drugą operację.

### Ćwiczenie 1: Podaj tłumaczenie terminu "prowadnice szaf metalowych" na język angielski. Opisz, z jakich narzędzi skorzystałaś/eś.

Odpowiedź: 
- Google tłumacz: metal cabinet guides, 
- Chat GPT: metal cabinet rails lub metal cabinet slides,
- PONS: metal cabinet guides




W dalszych ćwiczeniach skupimy się jednak na odszukaniu terminu specjalistycznego w tekście. W tym celu będą potrzebne dwie operacje:
1. Przygotowanie słownika specjalistycznego.
2. Detekcja terminologii przy użyciu przygotowanego słownika specjalistycznego.

Zajmijmy się najpierw krokiem nr 2 (gdyż jest prostszy). Rozważmy następujący tekst:

In [2]:
text = " For all Java programmers:"
text += " This section explains how to compile and run a Swing application from the command line."
text += " For information on compiling and running a Swing application using NetBeans IDE,"
text += " see Running Tutorial Examples in NetBeans IDE. The compilation instructions work for all Swing programs"
text += " — applets, as well as applications. Here are the steps you need to follow:"
text += " Install the latest release of the Java SE platform, if you haven't already done so."
text += " Create a program that uses Swing components. Compile the program. Run the program."

Załóżmy, że posiadamy następujący słownik:

In [28]:
dictionary = ['program', 'application', 'applet', 'compile']

### Ćwiczenie 2: Napisz program, który wypisze pozycje wszystkich wystąpień poszczególnych terminów specjalistycznych. Dla każdego terminu należy wypisać listę par (pozycja_startowa, pozycja końcowa).

In [34]:
import re

def terminology_lookup(text, dictionary):
    terminology_position = {term: [] for term in dictionary}

    for term in dictionary:
        pattern = re.compile(rf"\b{term}\b")
        for match in pattern.finditer(text):
            terminology_position[term].append((match.start(), match.end()))

    return terminology_position

print(terminology_lookup(text, dictionary))

{'program': [(468, 475), (516, 523), (533, 540)], 'application': [(80, 91), (164, 175)], 'applet': [], 'compile': [(56, 63)]}


Zwykłe wyszukiwanie w tekście ma pewne wady. Na przykład, gdy szukaliśmy słowa "program", złapaliśmy przypadkiem słowo "programmer". Złapaliśmy także słowo "programs", co jest poprawne, ale niepoprawnie podaliśmy jego pozycję w tekście.

Żeby poradzić sobie z tymi problemami, musimy wykorzystać techniki przetwarzania języka naturalnego. Wypróbujmy pakiet spaCy:

`pip3 install spacy`

oraz

`python3 -m spacy download en_core_web_sm`

In [4]:
import spacy
nlp = spacy.load("en_core_web_sm")

doc = nlp(text)

for token in doc:
    print(token.lemma_)

 
for
all
Java
programmer
:
this
section
explain
how
to
compile
and
run
a
swing
application
from
the
command
line
.
for
information
on
compile
and
run
a
swing
application
use
NetBeans
IDE
,
see
run
Tutorial
Examples
in
NetBeans
IDE
.
the
compilation
instruction
work
for
all
Swing
program
—
applet
,
as
well
as
application
.
here
be
the
step
you
need
to
follow
:
install
the
late
release
of
the
Java
SE
platform
,
if
you
have
not
already
do
so
.
create
a
program
that
use
swing
component
.
compile
the
program
.
run
the
program
.


Sukces! Nastąpił podział tekstu na słowa (tokenizacja) oraz sprowadzenie do formy podstawowej każdego słowa (lematyzacja).

### Ćwiczenie 3: Zmodyfikuj program z ćwiczenia 2 tak, aby zwracał również odmienione słowa. Na przykład, dla słowa "program" powinien znaleźć również "programs", ustawiając pozycje w tekście odpowiednio dla słowa "programs". Wykorzystaj właściwość idx tokenu.

In [35]:
def terminology_lookup(text, dictionary):
    doc = nlp(text)
    terminology_position = {term: [] for term in dictionary}

    for token in doc:
        for term in dictionary:
            if token.lemma_.lower().startswith(term.lower()):
                terminology_position[term].append((token.idx, token.idx + len(token.text)))
    
    return terminology_position

result = terminology_lookup(text, dictionary)
print(result)



{'program': [(14, 25), (291, 299), (468, 475), (516, 523), (533, 540)], 'application': [(80, 91), (164, 175), (322, 334)], 'applet': [(302, 309)], 'compile': [(56, 63), (134, 143), (504, 511)]}


Teraz czas zająć się problemem przygotowania słownika specjalistycznego. W tym celu napiszemy nasz własny ekstraktor terminologii. Wejściem do ekstraktora będzie tekst zawierający specjalistyczną terminologię. Wyjściem - lista terminów.

Przyjmijmy następujące podejście - terminami specjalistycznymi będą najcześćiej występujące rzeczowniki w tekście. Wykonajmy krok pierwszy:

### Ćwiczenie 4: Wypisz wszystkie rzeczowniki z tekstu. Wykorzystaj możliwości spaCy.

In [36]:
def get_nouns(text):
    doc = nlp(text)
    all_nouns = []
    for token in doc:
        if token.pos_ == "NOUN":
            #print(token.text)
            #print(token.lemma_)
            all_nouns.append(token.lemma_)
            #print(all_nouns)
    return all_nouns

text = " For all Java programmers:"
text += " This section explains how to compile and run a Swing application from the command line."
text += " For information on compiling and running a Swing application using NetBeans IDE,"
text += " see Running Tutorial Examples in NetBeans IDE. The compilation instructions work for all Swing programs"
text += " — applets, as well as applications. Here are the steps you need to follow:"
text += " Install the latest release of the Java SE platform, if you haven't already done so."
text += " Create a program that uses Swing components. Compile the program. Run the program."

all_nouns_in_text = get_nouns(text)
print(all_nouns_in_text)

['programmer', 'section', 'swing', 'application', 'command', 'line', 'information', 'swing', 'application', 'compilation', 'instruction', 'program', 'applet', 'application', 'step', 'release', 'platform', 'program', 'swing', 'component', 'program', 'program']


Teraz czas na podliczenie wystąpień poszczególnych rzeczowników. Uwaga - różne formy tego samego słowa zliczamy razem jako wystąpienia tego słowa (np. "program" i "programs"). Najwygodniejszą metodą podliczania jest zastosowanie tzw. tally (po polsku "zestawienie"). Jest to słownik, którego kluczem jest słowo w formie podstawowej, a wartością liczba wystąpień tego słowa, wliczając słowa odmienione. Przykład gotowego tally:

In [37]:
tally = {"program" : 4, "component" : 1}

### Ćwiczenie 5: Napisz program do ekstrakcji terminologii z tekstu według powyższych wytycznych.

In [43]:
from collections import Counter

def extract_terms(text):
    doc = nlp(text)
    all_nouns = []
    tally = {}
    for token in doc:
        if token.pos_ == "NOUN":
            #print(token.text)
            #print(token.lemma_)
            all_nouns.append(token.lemma_)
            #print(all_nouns)
            
    tally = Counter(all_nouns)
    tally_dict = dict(tally)
    return tally_dict

text = " For all Java programmers:"
text += " This section explains how to compile and run a Swing application from the command line."
text += " For information on compiling and running a Swing application using NetBeans IDE,"
text += " see Running Tutorial Examples in NetBeans IDE. The compilation instructions work for all Swing programs"
text += " — applets, as well as applications. Here are the steps you need to follow:"
text += " Install the latest release of the Java SE platform, if you haven't already done so."
text += " Create a program that uses Swing components. Compile the program. Run the program."

tally_counter = extract_terms(text)
print(tally_counter)


{'programmer': 1, 'section': 1, 'swing': 3, 'application': 3, 'command': 1, 'line': 1, 'information': 1, 'compilation': 1, 'instruction': 1, 'program': 4, 'applet': 1, 'step': 1, 'release': 1, 'platform': 1, 'component': 1}


### Ćwiczenie 6: Rozszerz powyższy program o ekstrację czasowników i przymiotników.

In [42]:
from collections import Counter

def extract_terms(text):
    doc = nlp(text)
    all_nouns = []
    all_verbs = []
    all_adjectives = []
    tally_nouns = {}
    tally_verbs = {}
    tally_adjectives = {}
    
    for token in doc:
        if token.pos_ == "NOUN":
            all_nouns.append(token.lemma_)
        elif token.pos_ == "VERB":
            all_verbs.append(token.lemma_)
        elif token.pos_ == "ADJ":
            all_adjectives.append(token.lemma_)
            
    tally_nouns = dict(Counter(all_nouns))
    tally_verbs = dict(Counter(all_verbs))
    tally_adjectives = dict(Counter(all_adjectives))
    
    return tally_nouns, tally_verbs, tally_adjectives


text = " For all Java programmers:"
text += " This section explains how to compile and run a Swing application from the command line."
text += " For information on compiling and running a Swing application using NetBeans IDE,"
text += " see Running Tutorial Examples in NetBeans IDE. The compilation instructions work for all Swing programs"
text += " — applets, as well as applications. Here are the steps you need to follow:"
text += " Install the latest release of the Java SE platform, if you haven't already done so."
text += " Create a program that uses Swing components. Compile the program. Run the program."

tally_n, tally_v, tally_a = extract_terms(text)
print(tally_n)
print(tally_v)
print(tally_a)


{'programmer': 1, 'section': 1, 'swing': 3, 'application': 3, 'command': 1, 'line': 1, 'information': 1, 'compilation': 1, 'instruction': 1, 'program': 4, 'applet': 1, 'step': 1, 'release': 1, 'platform': 1, 'component': 1}
{'explain': 1, 'compile': 3, 'run': 4, 'use': 2, 'see': 1, 'work': 1, 'need': 1, 'follow': 1, 'install': 1, 'do': 1, 'create': 1}
{'late': 1}
